In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from scripts import roc_auc_hlca, hlca_markers, roc_auc_crispr

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def gen_random_df(orig_df):
    random_df = orig_df.copy()
    for c in random_df.columns:
        l = list(range(random_df.shape[0]))
        np.random.shuffle(l)
        random_df[c] = l
    return random_df

## Task 1

In [5]:
adata = sc.read(f'/home/icb/yuge.ji/projects/feature-attribution-sc/datasets/scgen_norman19.h5ad')
adata

AnnData object with n_obs × n_vars = 69570 × 19018
    obs: 'guide_identity', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'guide_AHR', 'guide_ARID1A', 'guide_ARRDC3', 'guide_ATL1', 'guide_BAK1', 'guide_BCL2L11', 'guide_BCORL1', 'guide_BPGM', 'guide_C19orf26', 'guide_C3orf72', 'guide_CBFA2T3', 'guide_CBL', 'guide_CDKN1A', 'guide_CDKN1B', 'guide_CDKN1C', 'guide_CEBPA', 'guide_CEBPB', 'guide_CEBPE', 'guide_CELF2', 'guide_CITED1', 'guide_CKS1B', 'guide_CLDN6', 'guide_CNN1', 'guide_CNNM4', 'guide_COL1A1', 'guide_COL2A1', 'guide_CSRNP1', 'guide_DLX2', 'guide_DUSP9', 'guide_EGR1', 'guide_ELMSAN1', 'guide_ETS2', 'guide_FEV', 'guide_FOSB', 'guide_FOXA1', 'guide_FOXA3', 'guide_FOXF1', 'guide_FOXL2', 'guide_FOXO4', 'guide_GLB1L2', 'guide_HES7', 'guide_HK2', 'guide_HNF4A', 'guide_HOXA13', 'guide_HOXB9', 'guide_HOXC13', 'guide_IER5L', 'guide_IGDCC3', 'guide_IKZF3', 'guide_IRF1', 'guide_ISL2', 'guide_JUN', 'guide_KIAA1804', 'guide_KIF18B', 'guide_KIF2C', 'g

In [6]:
df = adata.to_df()

In [7]:
df['pert'] = adata.obs.perturbation_name

In [8]:
%%time
mean_df = df.groupby('pert').mean().T

CPU times: user 3.46 s, sys: 2 s, total: 5.46 s
Wall time: 5.47 s


In [9]:
mean_df.to_csv('outputs/random/task1_mean.csv')

In [32]:
roc_auc_crispr(mean_df)

0.8689830667503896

In [38]:
roc_auc_hlca(gen_random_df(mean_df))

0.4208690469930413

## Task 2

In [5]:
hlca_path = 'datasets/hlca.h5ad'  # old hlca
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 584944 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

Mean:

In [6]:
df = adata.to_df()

In [7]:
df['cell_type'] = adata.obs.scanvi_label

In [8]:
%%time
mean_df = df.groupby('cell_type').mean().T

CPU times: user 2.14 s, sys: 1.66 ms, total: 2.14 s
Wall time: 2.14 s


In [9]:
mean_df.to_csv('outputs/random/task2_mean.csv')

In [10]:
roc_auc_hlca(mean_df)

0.9418018954812546

In [12]:
roc_auc_hlca(gen_random_df(mean_df))

0.46977306894398

In [11]:
# missing columns
marker_dict = hlca_markers()
set(mean_df.columns) - set(marker_dict.keys())

{'Bronchial Vessel 1',
 'Bronchial Vessel 2',
 'KRT5- KRT17+ epithelial',
 'Megakaryocytes',
 'Neutrophilic',
 'Proliferating cells',
 'Squamous',
 'unlabeled'}